# Calculating the LSD profiles 

## 0 Packages and data

In [7]:
path='G:/Shared drives/BeStarsMiMeS/UpdatedFiles/' #Patrick's google file stream path

In [17]:

import LSDpy.lsdpy as lsdpy


In [18]:
import specpolFlow as pol
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.constants as const

In [19]:
from matplotlib.backends.backend_pdf import PdfPages

import pandas as pd
import itertools

### This gets rid of the warnings in the cells where it is creating more than 20 figures (i.e. 1.2)
#### Not needed for this notebook but could be useful in the future
#import warnings
#warnings.filterwarnings("default", message="More than 20 figures have been opened.")

In [20]:
import pandas as pd
sheet_id = '1M6y1Wnsrc-w5FjUMfKaSFa_-foIDAaMe8W4lYNWnWyk'
sheet_name = 'Stars'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
StarData=pd.read_csv(url)

sheet_name = 'Observations'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
Observations=pd.read_csv(url)

## 0.1 Demo with a single observation


In [21]:
#note: currently nonfunctional
#star = 'hd6226'
#obs = '1'
#modelcode = 'T15000G40'
# careful, check whether it is g or G in the spreadsheet....
#vsini = 120.0
#vradCorrected = -70.0 *u.km/u.s

#folderMask='depth0.1_tellClean'

#obsFolder='Synth-hybrid-spectra'
#obsnamecode = '_noise'

#lsdfolder = 'hybrid__maskdepth0.1_tellClean'

#mask = 'Masks/{}/{}_depth0.1.mask'.format(folderMask,modelcode)

#file = '{}/{}_{}{}.s'.format(obsFolder, star, obs, obsnamecode)
#outfile = 'LSD/{}/{}_{}.lsd'.format(lsdfolder, star,obs)


#toto = lsdpy.main(observation=file, mask=mask, outName=outfile, 
#           velStart=-400.0, velEnd=400.0, velPixel=2.6, 
#           normDepth=0.2, normLande=1.2, normWave=500.0,
#           removeContPol=1, trimMask=0, sigmaClipIter=0, sigmaClip=500, 
#           interpMode=1, fSaveModelS=0, outModelName='',
#           fLSDPlotImg=0, fSavePlotImg=0, outPlotImgName='')


In [22]:
#lsd = pol.iolsd.read_lsd(outfile)
#lsd.plot()

### 2.1 **Spectra**: Synth-hybrid-spectra
reran - Patrick


In [23]:
df = pd.read_csv('{}03-LSD-calculations/LSD/Params-LSDProfiles-CalculatedByPython.csv'.format(path))
blend=np.array(df['Name'][df['VelEnd']>=900])
blend

array(['hd20336', 'hd33328', 'hd45725', 'hd58978', 'hd77320', 'hd89884',
       'hd138749', 'hd164284', 'hd164906', 'hd191610', 'hd217050'],
      dtype=object)

In [24]:
Observations

,1,Unnamed: 1,Unnamed: 2,Name,LSDfileName,NameAsif,NameOfMask,Unnamed: 7,NameOfSymbolicLinkToData,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,185,1.0,1,hd6226,hd6226_1.lsd.s,hd6226_1,mask_T15000g40lamb3700_9000.dat,0,hd6226_11AP14_1310132pnv.s,110.34,-60.297909,-70.000000
1,193,2.0,1,hd7636,hd7636_1.lsd.s,hd7636_1,mask_T15000g40lamb3700_9000.dat,0,hd7636_11AP14_1314767pnv.s,230.00,0.000000,0.000000
2,194,2.0,2,hd7636,hd7636_2.lsd.s,hd7636_2,mask_T15000g40lamb3700_9000.dat,0,hd7636_11AP14_1316457pnv.s,230.00,0.000000,0.000000
3,2,3.0,1,hd10144,hd10144_1.lsd.s,hd10144_1,mask_T20000g35lamb3700_9000.dat,1,hd10144_187.D-0917A_2011-12-11_01_pnv.s,245.31,12.981952,12.981952
4,3,4.0,1,hd10516,hd10516_1.lsd.s,hd10516_1,mask_T25000g40lamb3700_9000.dat,0,hd10516_11AP14_1311181pnv.s,382.04,-60.990786,-60.990786
...,...,...,...,...,...,...,...,...,...,...,...,...
196,137,74.0,1,hd221507,hd221507_1.lsd.s,hd221507_1,mask_T10500g40lamb3700_9000.dat,1,hd221507_187.D-0917A_2012-07-19_01_pnv.s,20.00,0.000072,0.000072
197,140,75.0,1,hd224686,hd224686_1.lsd.s,hd224686_1,mask_T13000g40lamb3700_9000.dat,1,hd224686_187.D-0917A_2011-12-09_01_pnv.s,286.16,11.694301,11.694301
198,141,75.0,2,hd224686,hd224686_2.lsd.s,hd224686_2,mask_T13000g40lamb3700_9000.dat,1,hd224686_187.D-0917A_2011-12-12_01_pnv.s,286.16,8.724928,8.724928
199,145,76.0,1,hd239712,hd239712_1.lsd.s,hd239712_1,mask_T21000g40lamb3700_9000.dat,0,hd239712_11BP14_1331344pnv.s,304.86,-0.002959,-0.002959


#### **Mask**: tellClean Mask depth 0.02:

In [25]:
%%capture
with PdfPages('{}03-LSD-calculations/hybrid__maskdepth0.02_tellClean.pdf'.format(path)) as pdf:

  for i, ObsInfo in Observations.iterrows():
    star = ObsInfo["Name"]
    obs = ObsInfo["Unnamed: 2"]
    for j in range(0,len(StarData["Name"])):
      if StarData["Name"][j] == star:
        modelcode = StarData["ModelCode"][j].upper()
        if StarData['vsini-estimate'][j]=='very bad':
          vsini = StarData["Adopted-vsini"][j]
        else:
          vsini = StarData['vsini-estimate'][j]
        velS = df["VelStart"][j]
        velE = df["VelEnd"][j]
        velP = df["VelPixel"][j]

    for j in range(0,len(blend)):
      if blend[j]==star:
        mask = '{}02-Mask-calculations/MaskCleaning/{}_depth0.02.mask'.format(path,star)
        break
      if blend[j]!=star:
        mask = '{}02-Mask-calculations/MaskCleaning/depth0.02_tellClean/{}_depth0.02.mask'.format(path,modelcode)

    file = '{}01-Synth-calculations/Synth-hybrid-spectra/{}_{}_noise.s'.format(path,star, obs)
    outfile = '{}03-LSD-calculations/LSD/hybrid_maskdepth0.02_tellClean/{}_{}.lsd'.format(path,star,obs)

    toto = lsdpy.main(observation=file, mask=mask, outName=outfile, 
            velStart=velS, velEnd=velE, velPixel=velP, 
            normDepth=0.2, normLande=1.2, normWave=500.0,
            removeContPol=1, trimMask=0, sigmaClipIter=0, sigmaClip=500, 
            interpMode=1, fSaveModelS=0, outModelName='',
            fLSDPlotImg=0, fSavePlotImg=0, outPlotImgName='')
    lsd = pol.iolsd.read_lsd(outfile)
    
    fig, ax = plt.subplots(4,1, figsize=(10,10))

    ax[3].errorbar(lsd.vel, lsd.specI, yerr=lsd.specSigI, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[2].errorbar(lsd.vel, lsd.specN1, yerr=lsd.specSigN1, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[1].errorbar(lsd.vel, lsd.specN2, yerr=lsd.specSigN2, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[0].errorbar(lsd.vel, lsd.specV, yerr=lsd.specSigV, xerr=None, fmt='o', ms=3, ecolor='0.5',c='k')
    plt.subplots_adjust(hspace=.0)
    ax[3].set_xlabel('Velocity (km/s)')
    ax[3].set_ylabel('I')
    ax[2].set_ylabel('N1')
    ax[1].set_ylabel('N2')
    ax[0].set_ylabel('V')
    fig.suptitle(star + '-'+ str(obs),y=0.92)

    pdf.savefig(fig)

##~30min

#### **Mask**: tellClean Mask depth 0.1:

In [26]:
%%capture
with PdfPages('{}03-LSD-calculations/hybrid__maskdepth0.1_tellClean.pdf'.format(path)) as pdf:

  for i, ObsInfo in Observations.iterrows():
    star = ObsInfo["Name"]
    obs = ObsInfo["Unnamed: 2"]
    for j in range(0,len(StarData["Name"])):
      if StarData["Name"][j] == star:
        modelcode = StarData["ModelCode"][j].upper()
        if StarData['vsini-estimate'][j]=='very bad':
          vsini = StarData["Adopted-vsini"][j]
        else:
          vsini = StarData['vsini-estimate'][j]
        velS = df["VelStart"][j]
        velE = df["VelEnd"][j]
        velP = df["VelPixel"][j]

    for j in range(0,len(blend)):
      if blend[j]==star:
        mask = '{}02-Mask-calculations/MaskCleaning/{}_depth0.1.mask'.format(path,star)
        break
      if blend[j]!=star:
        mask = '{}02-Mask-calculations/MaskCleaning/depth0.1_tellClean/{}_depth0.1.mask'.format(path,modelcode)

    file = '{}01-Synth-calculations/Synth-hybrid-spectra/{}_{}_noise.s'.format(path,star, obs)
    outfile = '{}03-LSD-calculations/LSD/hybrid_maskdepth0.1_tellClean/{}_{}.lsd'.format(path,star,obs)

    toto = lsdpy.main(observation=file, mask=mask, outName=outfile, 
            velStart=velS, velEnd=velE, velPixel=velP, 
            normDepth=0.2, normLande=1.2, normWave=500.0,
            removeContPol=1, trimMask=0, sigmaClipIter=0, sigmaClip=500, 
            interpMode=1, fSaveModelS=0, outModelName='',
            fLSDPlotImg=0, fSavePlotImg=0, outPlotImgName='')
    lsd = pol.iolsd.read_lsd(outfile)
    
    fig, ax = plt.subplots(4,1, figsize=(10,10))

    ax[3].errorbar(lsd.vel, lsd.specI, yerr=lsd.specSigI, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[2].errorbar(lsd.vel, lsd.specN1, yerr=lsd.specSigN1, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[1].errorbar(lsd.vel, lsd.specN2, yerr=lsd.specSigN2, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[0].errorbar(lsd.vel, lsd.specV, yerr=lsd.specSigV, xerr=None, fmt='o', ms=3, ecolor='0.5',c='k')
    plt.subplots_adjust(hspace=.0)
    ax[3].set_xlabel('Velocity (km/s)')
    ax[3].set_ylabel('I')
    ax[2].set_ylabel('N1')
    ax[1].set_ylabel('N2')
    ax[0].set_ylabel('V')
    fig.suptitle(star + '-'+ str(obs),y=0.92)

    pdf.savefig(fig)

##~30min

#### **Mask**: tellClean CleanAsifTweak

In [27]:
%%capture
with PdfPages('{}03-LSD-calculations/hybrid_CleanAsifTweak_tellClean.pdf'.format(path)) as pdf:

  for i, ObsInfo in Observations.iterrows():
    star = ObsInfo["Name"]
    obs = ObsInfo["Unnamed: 2"]
    for j in range(0,len(StarData["Name"])):
      if StarData["Name"][j] == star:
        modelcode = StarData["ModelCode"][j].upper()
        vsini = StarData["Adopted-vsini"][j]
        velS = df["VelStart"][j]
        velE = df["VelEnd"][j]
        velP = df["VelPixel"][j]

    mask = '{}02-Mask-calculations/MaskCleaning/AsifMaskCleanTweak_tellClean/{}_clean_{}.mask_auto_twk'.format(path,star,obs)

    file = '{}01-Synth-calculations/Synth-hybrid-spectra/{}_{}_noise.s'.format(path,star, obs)
    outfile = '{}03-LSD-calculations/LSD/hybrid_CleanAsifTweak+tellClean/{}_{}.lsd'.format(path,star,obs)

    toto = lsdpy.main(observation=file, mask=mask, outName=outfile, 
            velStart=velS, velEnd=velE, velPixel=velP, 
            normDepth=0.2, normLande=1.2, normWave=500.0,
            removeContPol=1, trimMask=0, sigmaClipIter=0, sigmaClip=500, 
            interpMode=1, fSaveModelS=0, outModelName='',
            fLSDPlotImg=0, fSavePlotImg=0, outPlotImgName='')
    lsd = pol.iolsd.read_lsd(outfile)
    
    fig, ax = plt.subplots(4,1, figsize=(10,10))

    ax[3].errorbar(lsd.vel, lsd.specI, yerr=lsd.specSigI, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[2].errorbar(lsd.vel, lsd.specN1, yerr=lsd.specSigN1, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[1].errorbar(lsd.vel, lsd.specN2, yerr=lsd.specSigN2, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[0].errorbar(lsd.vel, lsd.specV, yerr=lsd.specSigV, xerr=None, fmt='o', ms=3, ecolor='0.5',c='k')
    plt.subplots_adjust(hspace=.0)
    ax[3].set_xlabel('Velocity (km/s)')
    ax[3].set_ylabel('I')
    ax[2].set_ylabel('N1')
    ax[1].set_ylabel('N2')
    ax[0].set_ylabel('V')
    fig.suptitle(star + '-'+ str(obs),y=0.92)

    pdf.savefig(fig)

##~30min

#### **Mask**: tellClean AsifClean

In [28]:
%%capture
with PdfPages('{}03-LSD-calculations/hybrid_CleanAsif_tellClean.pdf'format(path)) as pdf:

  for i, ObsInfo in Observations.iterrows():
    star = ObsInfo["Name"]
    obs = ObsInfo["Unnamed: 2"]
    for j in range(0,len(StarData["Name"])):
      if StarData["Name"][j] == star:
        modelcode = StarData["ModelCode"][j].upper()
        vsini = StarData["Adopted-vsini"][j]
        velS = df["VelStart"][j]
        velE = df["VelEnd"][j]
        velP = df["VelPixel"][j]

    mask = '{}02-Mask-calculations/MaskCleaning/AsifMaskClean_tellClean/{}_clean_{}.mask'.format(path,star,obs)

    file = '{}01-Synth-calculations/Synth-hybrid-spectra/{}_{}_noise.s'.format(path,star, obs)
    outfile = '{}03-LSD-calculations/LSD/hybrid_CleanAsif+tellClean/{}_{}.lsd'.format(path,star,obs)

    toto = lsdpy.main(observation=file, mask=mask, outName=outfile, 
            velStart=velS, velEnd=velE, velPixel=velP, 
            normDepth=0.2, normLande=1.2, normWave=500.0,
            removeContPol=1, trimMask=0, sigmaClipIter=0, sigmaClip=500, 
            interpMode=1, fSaveModelS=0, outModelName='',
            fLSDPlotImg=0, fSavePlotImg=0, outPlotImgName='')
    lsd = pol.iolsd.read_lsd(outfile)
    
    fig, ax = plt.subplots(4,1, figsize=(10,10))

    ax[3].errorbar(lsd.vel, lsd.specI, yerr=lsd.specSigI, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[2].errorbar(lsd.vel, lsd.specN1, yerr=lsd.specSigN1, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[1].errorbar(lsd.vel, lsd.specN2, yerr=lsd.specSigN2, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[0].errorbar(lsd.vel, lsd.specV, yerr=lsd.specSigV, xerr=None, fmt='o', ms=3, ecolor='0.5',c='k')
    plt.subplots_adjust(hspace=.0)
    ax[3].set_xlabel('Velocity (km/s)')
    ax[3].set_ylabel('I')
    ax[2].set_ylabel('N1')
    ax[1].set_ylabel('N2')
    ax[0].set_ylabel('V')
    fig.suptitle(star + '-'+ str(obs),y=0.92)

    pdf.savefig(fig)

##~30min

SyntaxError: invalid syntax (Temp/ipykernel_30768/4186271776.py, line 1)

### 2.2 **Spectra**: Normalized-Spectra
reran - Patrick


In [ ]:
df = pd.read_csv('{}03-LSD-calculations/LSD/Params-LSDProfiles-CalculatedByPython.csv'.format(path))
blend=np.array(df['Name'][df['VelEnd']>=900])
blend

#### **Mask**: tellClean Mask depth 0.02:

In [ ]:
%%capture

with PdfPages('{}03-LSD-calculations/normalized_maskdepth0.02_tellClean.pdf'.format(path)) as pdf:

  for i, ObsInfo in Observations.iterrows():
    star = ObsInfo["Name"]
    obs = ObsInfo["Unnamed: 2"]
    for j in range(0,len(StarData["Name"])):
      if StarData["Name"][j] == star:
        modelcode = StarData["ModelCode"][j].upper()
        if StarData['vsini-estimate'][j]=='very bad':
          vsini = StarData["Adopted-vsini"][j]
        else:
          vsini = StarData['vsini-estimate'][j]
        velS = df["VelStart"][j]
        velE = df["VelEnd"][j]
        velP = df["VelPixel"][j]

    for j in range(0,len(blend)):
      if blend[j]==star:
        mask = '{}02-Mask-calculations/MaskCleaning/{}_depth0.02.mask'.format(path,star)
        break
      if blend[j]!=star:
        mask = '{}02-Mask-calculations/MaskCleaning/depth0.02_tellClean/{}_depth0.02.mask'.format(path,modelcode)

    file = '{}00-InputMaterial/NormalizedSpectra/{}_{}.s'.format(path,star, obs)
    outfile = '{}03-LSD-calculations/LSD/normalized_maskdepth0.02_tellClean/{}_{}.lsd'.format(path,star,obs)

    toto = lsdpy.main(observation=file, mask=mask, outName=outfile, 
            velStart=velS, velEnd=velE, velPixel=velP, 
            normDepth=0.2, normLande=1.2, normWave=500.0,
            removeContPol=1, trimMask=0, sigmaClipIter=0, sigmaClip=500, 
            interpMode=1, fSaveModelS=0, outModelName='',
            fLSDPlotImg=0, fSavePlotImg=0, outPlotImgName='')
    lsd = pol.iolsd.read_lsd(outfile)
    
    fig, ax = plt.subplots(4,1, figsize=(10,10))

    ax[3].errorbar(lsd.vel, lsd.specI, yerr=lsd.specSigI, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[2].errorbar(lsd.vel, lsd.specN1, yerr=lsd.specSigN1, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[1].errorbar(lsd.vel, lsd.specN2, yerr=lsd.specSigN2, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[0].errorbar(lsd.vel, lsd.specV, yerr=lsd.specSigV, xerr=None, fmt='o', ms=3, ecolor='0.5',c='k')
    plt.subplots_adjust(hspace=.0)
    ax[3].set_xlabel('Velocity (km/s)')
    ax[3].set_ylabel('I')
    ax[2].set_ylabel('N1')
    ax[1].set_ylabel('N2')
    ax[0].set_ylabel('V')
    fig.suptitle(star + '-'+ str(obs),y=0.92)

    pdf.savefig(fig)

##~30min

#### **Mask**: tellClean Mask depth 0.1:

In [ ]:
%%capture

with PdfPages('{}03-LSD-calculations/normalized_maskdepth0.1_tellClean.pdf'.format(path)) as pdf:

  for i, ObsInfo in Observations.iterrows():
    star = ObsInfo["Name"]
    obs = ObsInfo["Unnamed: 2"]
    for j in range(0,len(StarData["Name"])):
      if StarData["Name"][j] == star:
        modelcode = StarData["ModelCode"][j].upper()
        if StarData['vsini-estimate'][j]=='very bad':
          vsini = StarData["Adopted-vsini"][j]
        else:
          vsini = StarData['vsini-estimate'][j]
        velS = df["VelStart"][j]
        velE = df["VelEnd"][j]
        velP = df["VelPixel"][j]

    for j in range(0,len(blend)):
      if blend[j]==star:
        mask = '{}02-Mask-calculations/MaskCleaning/{}_depth0.1.mask'.format(path,star)
        break
      if blend[j]!=star:
        mask = '{}02-Mask-calculations/MaskCleaning/depth0.1_tellClean/{}_depth0.1.mask'.format(path,modelcode)

    file = '{}00-InputMaterial/NormalizedSpectra/{}_{}.s'.format(path,star, obs)
    outfile = '{}03-LSD-calculations/LSD/normalized_maskdepth0.1_tellClean/{}_{}.lsd'.format(path,star,obs)

    toto = lsdpy.main(observation=file, mask=mask, outName=outfile, 
            velStart=velS, velEnd=velE, velPixel=velP, 
            normDepth=0.2, normLande=1.2, normWave=500.0,
            removeContPol=1, trimMask=0, sigmaClipIter=0, sigmaClip=500, 
            interpMode=1, fSaveModelS=0, outModelName='',
            fLSDPlotImg=0, fSavePlotImg=0, outPlotImgName='')
    lsd = pol.iolsd.read_lsd(outfile)
    
    fig, ax = plt.subplots(4,1, figsize=(10,10))

    ax[3].errorbar(lsd.vel, lsd.specI, yerr=lsd.specSigI, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[2].errorbar(lsd.vel, lsd.specN1, yerr=lsd.specSigN1, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[1].errorbar(lsd.vel, lsd.specN2, yerr=lsd.specSigN2, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[0].errorbar(lsd.vel, lsd.specV, yerr=lsd.specSigV, xerr=None, fmt='o', ms=3, ecolor='0.5',c='k')
    plt.subplots_adjust(hspace=.0)
    ax[3].set_xlabel('Velocity (km/s)')
    ax[3].set_ylabel('I')
    ax[2].set_ylabel('N1')
    ax[1].set_ylabel('N2')
    ax[0].set_ylabel('V')
    fig.suptitle(star + '-'+ str(obs),y=0.92)

    pdf.savefig(fig)

##~30min

#### **Mask**: tellClean CleanAsifTweak

In [ ]:
%%capture
with PdfPages('{}03-LSD-calculations/normalized_CleanAsifTweak_tellClean.pdf'.format(path)) as pdf:

  for i, ObsInfo in Observations.iterrows():
    star = ObsInfo["Name"]
    obs = ObsInfo["Unnamed: 2"]
    for j in range(0,len(StarData["Name"])):
      if StarData["Name"][j] == star:
        modelcode = StarData["ModelCode"][j].upper()
        vsini = StarData["Adopted-vsini"][j]
        velS = df["VelStart"][j]
        velE = df["VelEnd"][j]
        velP = df["VelPixel"][j]

    mask = '{}02-Mask-calculations/MaskCleaning/AsifMaskCleanTweak_tellClean/{}_clean_{}.mask_auto_twk'.format(path,star,obs)

    file = '{}00-InputMaterial/NormalizedSpectra/{}_{}.s'.format(path,star, obs)
    outfile = '{}03-LSD-calculations/LSD/normalized_CleanAsifTweak+tellClean/{}_{}.lsd'.format(path,star,obs)

    toto = lsdpy.main(observation=file, mask=mask, outName=outfile, 
            velStart=velS, velEnd=velE, velPixel=velP, 
            normDepth=0.2, normLande=1.2, normWave=500.0,
            removeContPol=1, trimMask=0, sigmaClipIter=0, sigmaClip=500, 
            interpMode=1, fSaveModelS=0, outModelName='',
            fLSDPlotImg=0, fSavePlotImg=0, outPlotImgName='')
    lsd = pol.iolsd.read_lsd(outfile)
    
    fig, ax = plt.subplots(4,1, figsize=(10,10))

    ax[3].errorbar(lsd.vel, lsd.specI, yerr=lsd.specSigI, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[2].errorbar(lsd.vel, lsd.specN1, yerr=lsd.specSigN1, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[1].errorbar(lsd.vel, lsd.specN2, yerr=lsd.specSigN2, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[0].errorbar(lsd.vel, lsd.specV, yerr=lsd.specSigV, xerr=None, fmt='o', ms=3, ecolor='0.5',c='k')
    plt.subplots_adjust(hspace=.0)
    ax[3].set_xlabel('Velocity (km/s)')
    ax[3].set_ylabel('I')
    ax[2].set_ylabel('N1')
    ax[1].set_ylabel('N2')
    ax[0].set_ylabel('V')
    fig.suptitle(star + '-'+ str(obs),y=0.92)

    pdf.savefig(fig)

##~30min

#### **Mask**: tellClean AsifClean

In [ ]:
%%capture
with PdfPages('{}03-LSD-calculations/normalized_CleanAsif_tellClean.pdf'.format(path)) as pdf:

  for i, ObsInfo in Observations.iterrows():
    star = ObsInfo["Name"]
    obs = ObsInfo["Unnamed: 2"]
    for j in range(0,len(StarData["Name"])):
      if StarData["Name"][j] == star:
        modelcode = StarData["ModelCode"][j].upper()
        vsini = StarData["Adopted-vsini"][j]
        velS = df["VelStart"][j]
        velE = df["VelEnd"][j]
        velP = df["VelPixel"][j]

    mask = '{}02-Mask-calculations/MaskCleaning/AsifMaskClean_tellClean/{}_clean_{}.mask'.format(path,star,obs)

    file = '{}00-InputMaterial/NormalizedSpectra/{}_{}.s'.format(path,star, obs)
    outfile = '{}03-LSD-calculations/LSD/normalized_CleanAsif+tellClean/{}_{}.lsd'.format(path,star,obs)

    toto = lsdpy.main(observation=file, mask=mask, outName=outfile, 
            velStart=velS, velEnd=velE, velPixel=velP, 
            normDepth=0.2, normLande=1.2, normWave=500.0,
            removeContPol=1, trimMask=0, sigmaClipIter=0, sigmaClip=500, 
            interpMode=1, fSaveModelS=0, outModelName='',
            fLSDPlotImg=0, fSavePlotImg=0, outPlotImgName='')
    lsd = pol.iolsd.read_lsd(outfile)
    
    fig, ax = plt.subplots(4,1, figsize=(10,10))

    ax[3].errorbar(lsd.vel, lsd.specI, yerr=lsd.specSigI, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[2].errorbar(lsd.vel, lsd.specN1, yerr=lsd.specSigN1, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[1].errorbar(lsd.vel, lsd.specN2, yerr=lsd.specSigN2, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[0].errorbar(lsd.vel, lsd.specV, yerr=lsd.specSigV, xerr=None, fmt='o', ms=3, ecolor='0.5',c='k')
    plt.subplots_adjust(hspace=.0)
    ax[3].set_xlabel('Velocity (km/s)')
    ax[3].set_ylabel('I')
    ax[2].set_ylabel('N1')
    ax[1].set_ylabel('N2')
    ax[0].set_ylabel('V')
    fig.suptitle(star + '-'+ str(obs),y=0.92)

    pdf.savefig(fig)

##~30min